In [6]:
import sys
import keras.layers

assert sys.version_info >= (3, 7)

In [7]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [8]:
from pathlib import Path

In [9]:
# define the default font sizes to make the figures prettier:

In [10]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100, kernel_initializer="lecun_normal",
                                    activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [11]:
dataset = tf.keras.datasets.cifar10.load_data()

In [12]:
optimizer = tf.keras.optimizers.legacy.Nadam(learning_rate=7e-4) #Legacy cause made on a m2
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [13]:
(X_train_full, y_train_full), (X_test, y_test) = dataset

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [14]:
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

In [16]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_Dropout_model.h5",
                                                         save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run__dropout{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [17]:
%load_ext tensorboard
%tensorboard --logdir=./my_cifar10_logs

Reusing TensorBoard on port 6006 (pid 10642), started 1:04:54 ago. (Use '!kill 10642' to kill it.)

In [18]:
model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 9s 5ms/step - loss: 1.9328 - accuracy: 0.3092 - val_loss: 1.7672 - val_accuracy: 0.3686
Epoch 2/100
  31/1407 [..............................] - ETA: 7s - loss: 1.7337 - accuracy: 0.3700

/opt/anaconda3/envs/homl3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 7s 5ms/step - loss: 1.7049 - accuracy: 0.3978 - val_loss: 1.6894 - val_accuracy: 0.3936
Epoch 3/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6176 - accuracy: 0.4285 - val_loss: 1.7534 - val_accuracy: 0.3942
Epoch 4/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.5563 - accuracy: 0.4545 - val_loss: 1.6389 - val_accuracy: 0.4330
Epoch 5/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.5059 - accuracy: 0.4747 - val_loss: 1.6664 - val_accuracy: 0.4462
Epoch 6/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4620 - accuracy: 0.4887 - val_loss: 1.5612 - val_accuracy: 0.4664
Epoch 7/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4251 - accuracy: 0.4991 - val_loss: 1.5950 - val_accuracy: 0.4654
Epoch 8/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.3837 - accuracy: 0.5157 - val_loss: 1.5203 - val_accuracy: 0.47

In [19]:
model.evaluate(X_valid_scaled, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.5203 - accuracy: 0.4778


[1.5203088521957397, 0.47780001163482666]